# 03 Tensorflow Lite Micro Deployment on Microcontroller Unit (MCU)

This notebook enables the deployment on the MCU by using the converted model (float32 or int8 model). It firsts converts the model into a C array, adds it to the Mbed-OS environment, and finally compiles the binary and flashes it on the MCU using the Mbed-OS toolchain.

In [ ]:
%run '00_README.ipynb'
%run 'H01_Models.ipynb'
%run 'H02_TFL-Conversion.ipynb'
%run 'H03_TFLu.ipynb'

In [ ]:
display(model_selection)

In [ ]:
# load model
tf_model_file = model_selection.value
tf_model = tf.keras.models.load_model(tf_model_file)

model_name = get_tf_model_string(tf_model_file)

In [ ]:
 data_selection = widgets.Dropdown(
    options=sorted(glob.glob("keras-model/*.py")),
    description='Select model:',
    layout=Layout(width='100%')
)
display(data_selection)

In [ ]:
tf_model_data = data_selection.value
%run -i {tf_model_data}


### Which converted model would you like to deploy?

In [ ]:
tfl_model_selection = widgets.Dropdown(
    options=glob.glob(f"TFLite-model/*{model_name}*.tflite"),
    description='Select model:',
)
display(tfl_model_selection)

# Deployment on a microcontroller (MCU)

The previously converted model can be deployed on a micro controller using [Tensorflow Lite Micro](https://www.tensorflow.org/lite/microcontrollers) (TFLu).

The TinyML community is the special interest group (SIG) within the TensorFlow community working on this. See their [charter](https://github.com/tensorflow/community/blob/master/sigs/micro/CHARTER.md).


The following example uses a generated `mbed-os` template including TFLu. For more information, please refer to the repositories:
- [TFLu_hello-world_mbed](https://gitlab.ethz.ch/tec/research/tensorflow/projects/ma_leheim/tflu_hello-world_mbed)
- [TFLu_hello-world_mbed_cmsis-nn](https://gitlab.ethz.ch/tec/research/tensorflow/projects/ma_leheim/tflu_hello-world_mbed_cmsis-nn)
- [TFLu_benchmark-model_mbed](https://gitlab.ethz.ch/tec/research/tensorflow/projects/ma_leheim/TFLu_benchmark-model_mbed)
- [TFLu_benchmark-model_mbed_cmsis-nn](https://gitlab.ethz.ch/tec/research/tensorflow/projects/ma_leheim/tflu_benchmark-model_mbed_cmsis-nn)


This requires [mbed-cli](https://github.com/ARMmbed/mbed-cli) and the [GNU Arm Embedded Toolchain](https://developer.arm.com/tools-and-software/open-source-software/developer-tools/gnu-toolchain/gnu-rm). Make sure to install them.

### Clone mbed benchmarking template

In [ ]:
display(mbed_selection)

In [ ]:
if mbed_selection.value == 'none':
    update_mbed_project(cmsis=False)
    mbed_dir = './TFLu_benchmark-model_mbed'
else:
    update_mbed_project()
    mbed_dir = './TFLu_benchmark-model_mbed_cmsis-nn'
    
    
mbed_non_cmsis_dir = './TFLu_benchmark-model_mbed'
mbed_cmsis_dir = './TFLu_benchmark-model_mbed_cmsis-nn'

mbed_dirs = [mbed_non_cmsis_dir, mbed_cmsis_dir]

### Deploy model to mbed

#### Write TF Lite model to the mbed project

The TF Lite model is a flatbuffers object which can be converted with `xxd -i` to an C array which can be interpreted by TFLu.

```
!xxd -i {file} > {file}.cc
```

In [ ]:
tfl_file = tfl_model_selection.value
tfl_model_to_file(tfl_file, mbed_dir)

### Copy a normalized example picture

A single image will be written as a constant array to the MCU which then can be used for inference on the device.

However, you also have the option to transfer images via UART for inference.

In [ ]:
display(inferences_slider)

In [ ]:
write_constants(tfl_file, inferences_slider.value, image_no, mbed_dir)

#### Build mbed project

For available macros please see the README of the mbed repository.

```
-N ARTIFACT_NAME, --artifact-name ARTIFACT_NAME
-c, --clean           Clean the build directory before compiling
-f, --flash           Flash the built firmware onto a connected target.
-D MACRO, --macro MACRO Add a macro definition
--profile PROFILE     Path of a build profile configuration file (or name of Mbed OS profile). Default: develop
```

In [ ]:
# fetch latest update and reset previous compilation settings
!cd {mbed_dir} && git fetch && git merge

In [ ]:
target_mcu = ''

interact(toggle_fpu, mbed_dir=fixed(mbed_dir), status=[('FPU enabled',1),('FPU disabled',0)])
interact(set_compiler_flag, mbed_dir=fixed(mbed_dir), flag=['-Ofast', '-Os'])
interact(patch_arena_size, mbed_dir=fixed(mbed_dir), size_kb=(0, 1000, 10))
# enter target devices of your choice
interact(set_target, target=['auto', 'NUCLEO_L496ZG', 'NUCLEO_F767ZI', '...'])

display(baudrate_slider)
display(cycles_selection, layers_selection, reporting_selection, input_selection, energy_selection)

In [ ]:
arguments = set_compilation_macros(INPUT_LENGTH, OUTPUT_LENGTH, baudrate=int(baudrate_slider.value),
           cycles=cycles_selection.value, layers=layers_selection.value, 
            reporting=reporting_selection.value, 
           manual_input=input_selection.value, energy=energy_selection.value)
print(arguments)

## Building and Flashing

In [ ]:
!cd {mbed_dir} && mbed compile \
-m {target_mcu} \
-t GCC_ARM \
--profile release \
--flash \
{arguments}

### Flash binary to STM32

If the automatic flashing of Mbed doesn't work:

```bash
cp {mbed_dir}/BUILD/NUCLEO_L496ZG/GCC_ARM-RELEASE/TFLu_hello-world_mbed_cmsis-nn.bin /Volumes/NODE_L496Z

# sometimes access to the volume is not permitted, then use st-flash
st-flash write {mbed_dir}/BUILD/NUCLEO_L496ZG/GCC_ARM/TFLu_hello-world_mbed_cmsis-nn.bin 0x8000000
```